In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap

# Add parent directory to sys.path in order to find common module
import sys
sys.path.insert(0,'../')

from common.datautils import load_ce_results

In [ ]:
average_data = load_ce_results('CE.results.zip')
average_data.head()

In [ ]:
average_data[average_data["Flags"] == '-O3']

In [ ]:
benchmarks = average_data["Benchmark"].unique()
print(benchmarks)

### Find best configuration for Energy

In [ ]:
def best_configuration(variable, benchmark, average_data):
    benchmark_data = average_data.loc[average_data["Benchmark"] == benchmark]
    min_index = benchmark_data[variable].idxmin()
    return benchmark_data.loc[min_index]["Flags"]
    

In [ ]:
for benchmark in benchmarks:
    print(benchmark)
    print(best_configuration('Energy', benchmark, average_data))
    print('-----')

### Compare Energy vs Time

In [ ]:
def plot_energy_vs_time(benchmark, data):
    benchmark_data = data.loc[data["Benchmark"] == benchmark]
    o3_data = benchmark_data.loc[benchmark_data["Flags"] == "-O3"]
    non_o3_data = benchmark_data.loc[benchmark_data["Flags"] != "-O3"]
    
    X = non_o3_data["Time"].values
    Y = non_o3_data["Energy"].values
    
    plt.figure()
    plt.scatter(X, Y, label='CE Configurations')
    plt.scatter(o3_data["Time"].values, o3_data["Energy"].values, c='r', label='-O3')
    plt.title('Energy vs Time results for CE on ' + benchmark)
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')

In [ ]:
for benchmark in benchmarks:
    plot_energy_vs_time(benchmark, average_data)

### Compare Energy vs Runs

In [ ]:
def plot_energy_vs_runs(benchmark, data):
    benchmark_data = data.loc[(data["Benchmark"] == benchmark) & ((data["Type"] == 'baseline') | (data["Type"] == 'initial'))].sort_values('RunId')
    
    X = benchmark_data["RunId"].values
    Y = benchmark_data["Energy"].values
    
    plt.figure()
    plt.plot(X, Y)
    plt.title('Energy vs run id for Combined Elimination on ' + benchmark)
    plt.xlabel('Run Id')
    plt.ylabel('Energy (J)')

In [ ]:
for benchmark in benchmarks:
    plot_energy_vs_runs(benchmark, average_data)

### Compare best improvement of Energy and time to O3 baseline

In [ ]:
def o3_relative_data(average_data, benchmarks):
    o3_data = average_data.loc[average_data["Flags"] == "-O3"]
    
    relative_data = average_data.copy(deep=True)
    
    for benchmark in benchmarks:
        o3 = o3_data.loc[relative_data["Benchmark"] == benchmark]
        o3_energy = o3.iloc[0]["Energy"]
        o3_time = o3.iloc[0]["Time"]

        relative_data.loc[relative_data["Benchmark"] == benchmark, "Energy"] /= o3_energy
        relative_data.loc[relative_data["Benchmark"] == benchmark, "Time"] /= o3_time

    return relative_data


In [ ]:
relative_data = o3_relative_data(average_data, benchmarks)

In [ ]:
def plot_relative_to_o3(variable, relative_data, benchmarks):
    X = benchmarks
    Y = [relative_data.loc[relative_data["Benchmark"] == benchmark, variable].min() for benchmark in benchmarks]

    plt.figure()
    plt.scatter(X, Y, marker='x', label='CE')
    plt.axhline(1, label='O3')
    
    plt.title('Best improvement of ' + variable + ' relative to -O3 found be CE.')
    plt.ylabel(variable + ' (relative to -O3)')
    plt.legend()
    
def double_plot_relative_to_o3(relative_data, benchmarks):
    X = benchmarks
    energy = [relative_data.loc[relative_data["Benchmark"] == benchmark, 'Energy'].min() for benchmark in benchmarks]
    time = [relative_data.loc[relative_data["Benchmark"] == benchmark, 'Time'].min() for benchmark in benchmarks]
    

    plt.figure()
    plt.scatter(X, energy, marker='o', label='Energy')
    plt.scatter(X, time, marker='x', label='Energy')
    
    plt.axhline(1, label='O3')
    
    plt.title('\n'.join(wrap('Best improvement of Energy and Time relative to -O3 found be CE.', 55)))
    plt.ylabel('Relative to -O3)')
    plt.xticks(rotation=90)
    plt.legend()

In [ ]:
double_plot_relative_to_o3(relative_data, benchmarks)

In [ ]:
plot_relative_to_o3('Energy', relative_data, benchmarks)
plot_relative_to_o3('Time', relative_data, benchmarks)